## ひろしまQuest2020#stayhome【コース予測部門】

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 100)

In [2]:
ALL_MERGE = 'intermediate/all_merge_15.f'
SUBMIT = 'submit/course_15.csv'

In [3]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 153)

In [4]:
print(all_pitch.shape)
column_cnt = len(all_pitch.columns)
all_pitch.head()

(778767, 153)


,No,ball,pitch_area,No_in_game,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,pitch_LR,bat_LR,bc_straight,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball,bc_area0,bc_area1,bc_area2,bc_area3,bc_area4,bc_area5,bc_area6,bc_area7,bc_area8,bc_area9,bc_area10,bc_area11,bc_area12,first,second,third,base_cnt,top_bot,role,pos_pit,date_from_opening,...,weight_bat,draft_order_bat,salary_bat,foreigner_bat,company_bat,univ_bat,highsch_bat,age_bat,play_year_bat,salary_year_bat,salary_x_year_bat,bmi_bat,batter_cnt,bat_game_cnt,firm,height,weight,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_year_dif_p-b,salary_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,bmi_dif_b-c,pit_bat_L_L,pit_bat_L_R,pit_bat_R_L,pit_bat_R_R
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0.0,1.0,0.464734,0.110359,0.151086,0.097383,0.067492,0.043210,0.012933,0.052803,0.064837,0.095199,0.082009,0.040512,0.063252,0.053916,0.030534,0.046508,0.038328,0.163762,0.057728,0.130872,0.132543,0,0,0,0,1,1,0,0,...,74,5.0,15000,0,1,1,0,32,8.0,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725,0,0,1,0
1,1,0.0,8.0,2,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0.0,1.0,0.442351,0.093302,0.109348,0.078518,0.141711,0.065447,0.017669,0.051654,0.044893,0.071306,0.068422,0.032002,0.043180,0.036149,0.029388,0.042910,0.033625,0.152078,0.078157,0.203642,0.164248,0,0,0,0,1,1,0,0,...,74,5.0,15000,0,1,1,0,32,8.0,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725,0,0,1,0
2,2,0.0,5.0,3,3,1,1,3,1,1,3,3,1,1,0,0,0,0,2,0.0,1.0,0.504212,0.069963,0.065568,0.046520,0.211722,0.059158,0.019231,0.023626,0.027656,0.045421,0.045238,0.025275,0.023810,0.027106,0.023077,0.032784,0.022711,0.202930,0.081319,0.285165,0.157509,0,0,0,0,1,1,0,0,...,74,5.0,15000,0,1,1,0,32,8.0,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725,0,0,1,0
3,3,0.0,12.0,4,4,1,2,1,1,2,4,4,2,1,0,0,1,0,0,0.0,0.0,0.475961,0.101792,0.226048,0.066617,0.042131,0.018345,0.003343,0.065762,0.034319,0.043647,0.027090,0.054336,0.065374,0.045046,0.062653,0.097944,0.085507,0.098410,0.098527,0.055307,0.231840,0,0,0,0,1,1,0,0,...,74,1.0,7040,0,0,0,1,40,22.0,320.0,154880.0,22.839506,268.0,81.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,3.191314,19000,2.0,1,2750.0,124000.0,-0.165367,2040,18.0,12,-930.0,134880.0,-3.356681,0,0,0,1
4,4,0.0,8.0,5,5,1,2,2,1,2,5,5,2,1,0,0,1,0,1,0.0,0.0,0.463496,0.099763,0.194714,0.067542,0.091198,0.023901,0.004486,0.054898,0.024390,0.034832,0.025124,0.039481,0.047557,0.033526,0.049596,0.080512,0.069337,0.085243,0.121054,0.076923,0.312424,0,0,0,0,1,1,0,0,...,74,1.0,7040,0,0,0,1,40,22.0,320.0,154880.0,22.839506,268.0,81.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,3.191314,19000,2.0,1,2750.0,124000.0,-0.165367,2040,18.0,12,-930.0,134880.0,-3.356681,0,0,0,1


### train
- 行数: 257117

In [5]:
train = all_pitch.dropna(subset=['pitch_area'])
train.shape

(257117, 153)

### test
- 行数: 521650

In [6]:
test = all_pitch[all_pitch['pitch_area'].isnull()]
test.shape

(521650, 153)

In [7]:
del all_pitch
gc.collect()

60

In [8]:
train_d = train.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

In [9]:
test_d = test.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

## Dataset作成

In [10]:
lgb_train = lgb.Dataset(train_d, train['pitch_area'])

### LGBM学習
- multiclass : クラス数=8
- tuning結果のパラメータを使用
- 'learning_rate' : 0.1

In [11]:
%%time
lgb_param = {
        'objective' : 'multiclass',
        'boosting_type': 'gbdt',
        'metric' : 'multi_logloss',
        'num_class' : 13,
        'seed' : 0,
        'learning_rate' : 0.1,
        'lambda_l1': 8.769293390201968, 
        'lambda_l2': 3.913949617576324e-05, 
        'num_leaves': 6, 
        'feature_fraction': 0.4, 
        'bagging_fraction': 0.8391111798378441, 
        'bagging_freq': 4, 
        'min_child_samples': 50
    }

# 学習
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=15000,
                    early_stopping_rounds=100,
                    verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['multi_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
best_cv_score = cv_results['multi_logloss-mean'][-1]
print('Best CV score:', best_cv_score)

[100]	cv_agg's multi_logloss: 2.35151 + 0.000690817
[200]	cv_agg's multi_logloss: 2.34534 + 0.000794651
[300]	cv_agg's multi_logloss: 2.34276 + 0.000843035
[400]	cv_agg's multi_logloss: 2.34121 + 0.000951219
[500]	cv_agg's multi_logloss: 2.34017 + 0.000985616
[600]	cv_agg's multi_logloss: 2.33956 + 0.00104864
[700]	cv_agg's multi_logloss: 2.33916 + 0.00111711
[800]	cv_agg's multi_logloss: 2.33884 + 0.00113938
[900]	cv_agg's multi_logloss: 2.33878 + 0.000997146
[1000]	cv_agg's multi_logloss: 2.33878 + 0.00101436
Best num_boost_round: 932
Best CV score: 2.3387434142884445
CPU times: user 1h 17min 6s, sys: 16 s, total: 1h 17min 22s
Wall time: 9min 41s


In [12]:
best_iter = int(num_boost_round * 1.1)

### 全体で再学習

In [13]:
%%time
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

CPU times: user 23min 45s, sys: 5.24 s, total: 23min 50s
Wall time: 2min 58s


### Feature Importance

In [14]:
fi = lgb_model.feature_importance()
fn = lgb_model.feature_name()
df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
df_feature_importance.sort_values('feat_imp', inplace=True)
df_feature_importance.tail(30)

,feat_name,feat_imp
132,salary_x_year_dif_p-b,793
128,salary_dif_p-b,804
85,salary_x_year_pit,807
8,pitcher_cnt_in_inning,815
91,changeup,822
84,salary_year_pit,833
0,No_in_game,839
86,bmi_pit,846
96,pit_inning_cnt,850
131,salary_year_dif_p-b,853


In [15]:
%%time
predict = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 6min 12s, sys: 748 ms, total: 6min 13s
Wall time: 48.1 s


## 結果出力

In [16]:
submit = pd.DataFrame(predict)
submit.reset_index(inplace=True)
print(submit.shape)
submit.head()

(521650, 14)


,index,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0,0.061637,0.146804,0.060507,0.041062,0.068482,0.039420,0.053846,0.055672,0.030812,0.190841,0.086688,0.084293,0.079938
1,1,0.051432,0.145247,0.083524,0.036489,0.054473,0.041416,0.041218,0.071699,0.034788,0.162076,0.072764,0.100168,0.104705
2,2,0.031237,0.082389,0.058024,0.024143,0.034402,0.031085,0.039199,0.050827,0.025500,0.191083,0.096379,0.199176,0.136556
3,3,0.036754,0.104725,0.058993,0.028664,0.035363,0.029557,0.042016,0.055247,0.031195,0.209545,0.107941,0.126751,0.133249
4,4,0.043736,0.127940,0.065347,0.033393,0.038840,0.049313,0.044194,0.066626,0.035281,0.186595,0.072950,0.113383,0.122402


In [17]:
submit.mean()

index    260824.500000
0             0.048700
1             0.069691
2             0.055994
3             0.046700
4             0.060083
5             0.046720
6             0.043141
7             0.065338
8             0.053029
9             0.111479
10            0.088227
11            0.133414
12            0.177485
dtype: float64

In [18]:
submit.to_csv(SUBMIT, header=False, index=False)

### SIGNATE

In [22]:
'signate submit --competition-id=276 ./{} --note feat={}_cv={}'.format(SUBMIT, column_cnt, best_cv_score)

'signate submit --competition-id=276 ./submit/course_15.csv --note feat=153_cv=2.3387434142884445'

### GCS

In [21]:
'gsutil cp ./{} gs://yasunobu_baseball/submit/'.format(SUBMIT)

'gsutil cp ./submit/course_15.csv gs://yasunobu_baseball/submit/'

### 評価結果
#### 15(153)  ...左右で球種/コースを別に集計
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 932
- Best CV score: 2.3387434142884445
- Wall time: 9min 41s
- 評価結果  : 2.3552543

#### 14(153)  ...左右で球種を別に集計
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 909
- Best CV score: 2.340082741044867
- Wall time: 9min 40s
- 評価結果  : 2.3555548

#### 3(157)  ...前の投球からの時間差を追加
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 857
- Best CV score: 2.340703597440808
- Wall time: 9min 20s
- 評価結果  : 2.3554087

#### 2(156)  ...merge12のパラメータをチューニング
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 835
- Best CV score: 2.3419206656914
- Wall time: 9min 7s
- 評価結果  : 2.3555299

#### 2(156)  ...merge12(2017のコースデータを付加) 
- 'learning_rate' : 0.1,
- Best num_boost_round: 121
- Best CV score: 2.344104226715559
- Wall time: 12min 30s
- 評価結果  : 2.3564326

#### 1(143)  ...球種予測の11と同じデータ 
- 'learning_rate' : 0.1,
- Best num_boost_round: 122
- Best CV score: 2.3445089434874253
- Wall time: 12min 2s (local)
- 評価結果  : 2.3562648